## Demo file on training the auto-encoder model

In [1]:
import numpy as np

In [2]:
a = np.load("sample.npy",allow_pickle=True)

In [22]:
### trim the data since each point cloud has different shape
sample = np.asarray([ t[:12100].reshape(-1,) for t in a])

In [23]:
from autoencode import Auto_Encoder
import torch
import torchvision
from torch import nn, optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader
import os

In [24]:
learning_rate = 0.0003
batch_size = 64
epochsize = 30

In [25]:
AE = Auto_Encoder()
MSELoss = nn.MSELoss()
optimizer = optim.Adam(AE.parameters(), lr=learning_rate)

In [26]:
try:
    sample = torch.from_numpy(sample)
except:
    pass
sample = sample.to(torch.float32)

for s in sample:
    
    s_prime = AE(s)

    # loss 
    loss = MSELoss(s_prime, s)

    # update
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [16]:
for i in range(10):
    for s in sample:
    
        s_prime = AE(s)

        # loss 
        loss = criteon(s_prime, s)

        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [27]:
AE(sample[0]) - sample[0]

tensor([ 0.9106, -0.2000,  0.0767,  ...,  0.1550, -0.1965,  0.0605],
       grad_fn=<SubBackward0>)

In [28]:
np.allclose(AE(sample[0]).detach().numpy()  , sample[0].detach().numpy() )

False